
# Lab 4: Using Access Security Settings and Policies 
## Applying Access Control Expressions (ACE) to Volumes Using ACE's and Global Security Policies 

## Security on Ezmeral Data Fabric.

Security on the Ezmearl Data Fabric is applied at the Volume which can be applied to:

- Directories and files
- Tables (if you have the full features Ezmeral Data Fabric)
- Streams (if you have the full features Ezmeral Data Fabric)

Security can be applied in two ways:

- On-demand per data type by creating an Access Control Experssion (ACE) and then applying that ACE as a security tag to the volume.
- Or through a local or global policy management which creates a catalog of security tags that can then be applied to any data type.

Below is an example we will be running through for this lab:

<p align="center">
  <img src="Pictures/edf-sec-ace-example.png" width="800" height="600" alt="Data Fabric Snapshots">                                                    
  
</p>

For more information on security features within the Ezmeral Data Fabric please see [Managing Security Policies](https://docs.datafabric.hpe.com/62/SecurityGuide/ManagingSecurityPolicies.html)

In [ ]:
%login {{ hostvars[inventory_hostname]['IP-WKSHP-Data-Fabric101'] }}

echo "We are now connected to the appliance"

#Variables used by scripts
#password="{{ PASSSTU }}" # your Data Fabric login credentials - password - 
#apiserver_host="{{ DATAFABRICAPISERVERIP }}" # API Server for the Data Fabric Cluster
#ad_group="{{ DATAFABRICADGROUP }}" # LDAP Domain Group for Data Fabric users
#vol_name="{{ DATAFABRICVOL }}" # suffix for student volume name
#snapvol_name="{{ DATAFABRICSNAPVOL }}" # suffix for student snap volume name
#cluster_name="{{ DATAFABRICCLUSTERAME }}" # Data Fabric Cluster Name
#src_vol_name="{{ DATAFABRICVOLSRCVOLNAME }}" # suffix for student source mirror volume name
#dst_vol_name="{{ DATAFABRICVOLDSTVOLNAME }}"  # suffix for student destination mirror volume name
#secvol_name="{{ DATAFABRICSECVOL }}"  # suffix for student volume name used for security comparison / user access 
#sec_policy="student{{ STDID }}-policy" #security policy to allow r/w access to volumes

echo "your operation context is:" student{{ STDID }} "with Password:" {{ PASSSTU }}
echo "----"
echo "your test user for applying security policy context is:" sec-student{{ STDID }} "with Password:" {{ PASSSTU }}

## 1. Create a volume that we can apply security policies.

#### Background:

You have been given a user that is allowed to create and modify volumes on the cluster and another user to test your security policies. Data Fabric volumes can be mounted so they are accessible from NFS, HDFS or POSIX commands (like the bash kernel shell in these labs). For security reasons, you can only mount volumes in the /user directory.

In this lab example we will:

1.	Created a new volume
2.	Tested writing and reading data to it from multiple users.
3.	Applied a ‘read only’ Access Control Expression to the volume.
4.	Tested writing and reading data to it from the sec-studentXXXX user.
5.	Created a security policy with read and write Access Control Expression’s.
6.	Apply the security policy to the volume.
7.	Tested writing and reading data to it from the sec-studentXXXX user.

To understand how an ACE works on a volume with files and directories first let's create a volume that all users can write files to:


In [ ]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/create?name='$username-$secvol_name'&path=/user/'$username-$secvol_name'&rootdirgroup=Data-Fabric-Users&rootdirperms=777' --user $username:$password

## 2. Test that both users can read/write data to the created volume

- We will list out the files and create/append data to a file as the username:

You are logging in as the user:
<p align="center">
  <img src="Pictures/studentxxxx.png" alt="studentxxx-Admin">                                                    
  
</p>

In [ ]:
source ~/.bashrc
echo "##Verify which user we are curently logged in as.##"
echo $username

In [ ]:
source ~/.bashrc
echo "##Create a new file on our security volume.##"
echo "$username-$secvol_name security volume update $(date)" >> /mapr/$cluster_name/user/$username-$secvol_name/$username-security-volume-test1
echo "##List out the volume directory.##"
ls /mapr/$cluster_name/user/$username-$secvol_name
echo "##Verify the file contents we just wrote.##"
cat /mapr/$cluster_name/user/$username-$secvol_name/$username-security-volume-test1

In [ ]:
%logout

- Next we will verify that another user can list out the files and create/append data to a file as the username:

You are logging in as the user:
<p align="center">
  <img src="Pictures/sec-studentxxxx.png" alt="studentxxx-Dev">                                                    
  
</p>

In [ ]:
%login sec-appliance

cat > $HOME/.bashrc << EOF
username="student{{ STDID }}" # your Data Fabric login credentials - username  
password="{{ PASSSTU }}" # your Data Fabric login credentials - password - 
sec_username="sec-student{{ STDID }}" # your Data Fabric login credentials - sec username  
sec_password="stuDISCO2020" # your Data Fabric login credentials - sec password - 
apiserver_host="{{ DATAFABRICAPISERVERIP }}" # API Server for the Data Fabric Cluster
ad_group="{{ DATAFABRICADGROUP }}" # LDAP Domain Group for Data Fabric users
vol_name="{{ DATAFABRICVOL }}" # suffix for student volume name
snapvol_name="{{ DATAFABRICSNAPVOL }}" # suffix for student snap volume name
cluster_name="{{ DATAFABRICCLUSTERAME }}" # Data Fabric Cluster Name
src_mirrvol1="{{ DATAFABRICVOLSRCVOLNAME }}" # suffix for student source mirror volume name
dst_mirrvol1="{{ DATAFABRICVOLDSTVOLNAME }}"  # suffix for student destination mirror volume name
secvol_name="{{ DATAFABRICSECVOL }}"  # suffix for student volume name used for security comparison / user access 

EOF


source ~/.bashrc
echo "login as sec-student{{ STDID }}"
id
echo $sec_username

In [ ]:
source ~/.bashrc
echo "##Create a new file as a different user on our security volume.##"
echo "$sec_username-$secvol_name security volume sec user update $(date)" >> /mapr/$cluster_name/user/$username-$secvol_name/$username-security-volume-secuser-test1
echo "##List out the volume directory.##"
ls /mapr/$cluster_name/user/$username-$secvol_name/
echo "##Verify the file contents we just wrote as a new user.##"
cat /mapr/$cluster_name/user/$username-$secvol_name/$username-security-volume-secuser-test1


In [ ]:
%logout

## 3. Modify a volume and apply a read Access Control Expression (ACE)

A valid user or group must be present on your cluster or in your directory services.  In this case we will use the following user to apply a read only ACE to the volume:

You are logging in as the user:
<p align="center">
  <img src="Pictures/studentxxxx.png" alt="studentxxx-Admin">                                                    
  
</p>


In [ ]:
%login {{ hostvars[inventory_hostname]['IP-WKSHP-Data-Fabric101'] }}

echo "We are now connected to the appliance"

#Variables used by scripts
#password="{{ PASSSTU }}" # your Data Fabric login credentials - password - 
#apiserver_host="{{ DATAFABRICAPISERVERIP }}" # API Server for the Data Fabric Cluster
#ad_group="{{ DATAFABRICADGROUP }}" # LDAP Domain Group for Data Fabric users
#vol_name="{{ DATAFABRICVOL }}" # suffix for student volume name
#snapvol_name="{{ DATAFABRICSNAPVOL }}" # suffix for student snap volume name
#cluster_name="{{ DATAFABRICCLUSTERAME }}" # Data Fabric Cluster Name
#src_vol_name="{{ DATAFABRICVOLSRCVOLNAME }}" # suffix for student source mirror volume name
#dst_vol_name="{{ DATAFABRICVOLDSTVOLNAME }}"  # suffix for student destination mirror volume name
#secvol_name="{{ DATAFABRICSECVOL }}"  # suffix for student volume name used for security comparison / user access 
#sec_policy="student{{ STDID }}-policy" #security policy to allow r/w access to volumes

echo "your operation context is:" student{{ STDID }} "with Password:" {{ PASSSTU }}

In [ ]:
source ~/.bashrc
echo "## We will now apply an ACE to the security volume, in this example the new user will only be allowed 'Read Only' access. ##"
curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/modify?name='$username-$secvol_name'&writeAce=u%3A'$username'&readAce=u%3A'$sec_username'' --user $username:$password

In [ ]:
%logout

## 4. Verify the security policy change.

We will now verify that the previous step we applied only has read-only access and is unable to write any data to the directory or file for the user:

You are logging in as the user:
<p align="center">
  <img src="Pictures/sec-studentxxxx.png" alt="studentxxx-Dev">                                                    
  
</p>

In [ ]:
%login sec-appliance

cat > $HOME/.bashrc << EOF
username="student{{ STDID }}" # your Data Fabric login credentials - username  
password="{{ PASSSTU }}" # your Data Fabric login credentials - password - 
sec_username="sec-student{{ STDID }}" # your Data Fabric login credentials - sec username  
sec_password="stuDISCO2020" # your Data Fabric login credentials - sec password - 
apiserver_host="{{ DATAFABRICAPISERVERIP }}" # API Server for the Data Fabric Cluster
ad_group="{{ DATAFABRICADGROUP }}" # LDAP Domain Group for Data Fabric users
vol_name="{{ DATAFABRICVOL }}" # suffix for student volume name
snapvol_name="{{ DATAFABRICSNAPVOL }}" # suffix for student snap volume name
cluster_name="{{ DATAFABRICCLUSTERAME }}" # Data Fabric Cluster Name
src_mirrvol1="{{ DATAFABRICVOLSRCVOLNAME }}" # suffix for student source mirror volume name
dst_mirrvol1="{{ DATAFABRICVOLDSTVOLNAME }}"  # suffix for student destination mirror volume name
secvol_name="{{ DATAFABRICSECVOL }}"  # suffix for student volume name used for security comparison / user access 
sec_policy="student{{ STDID }}-policy" #security policy to allow r/w access to volumes
EOF

echo "login as sec-student{{ STDID }}"
id

In [ ]:
source ~/.bashrc
echo "## Make an update to the file as the new user. ##"
echo "$sec_username-$secvol_name security volume sec user update $(date)" >> /mapr/$cluster_name/user/$username-$secvol_name/$username-security-volume-secuser-test1
echo "## List out the volume diretory to the files. ##"
ls /mapr/$cluster_name/user/$username-$secvol_name/
echo "## Verify the update to the file as the new user. ##"
cat /mapr/$cluster_name/user/$username-$secvol_name/$username-security-volume-secuser-test1
echo "##"
echo "## You should have recieved a 'Pemission denied', this is expected as the new user only has 'Read only' access now to the volume. ##"


In [ ]:
%logout

## 5. Creating Security Policies

#### Background

Security polices allow you to create a central policy that can then be applied to multiple data objects.  In this case a policy can have an Access Control Expression (ACE) for a set of users or groups and you then apply that policy as a "tag" to your volume.

We will first create a security policy for the authentication services group that has all of our students in ito the directory or file for the user:


You are logging in as the user:
<p align="center">
  <img src="Pictures/studentxxxx.png" alt="studentxxx-Admin">                                                    
  
</p>



In [ ]:
%login {{ hostvars[inventory_hostname]['IP-WKSHP-Data-Fabric101'] }}

In [ ]:
source ~/.bashrc
echo "## Create a new security policy with write and read access and add the new user to it. ##"
curl -k -X POST 'https://'$apiserver_host':8443/rest/security/policy/create?name='$sec_policy'&allowtagging=true&accesscontrol=Armed&wiresecurityenabled=true&readaces=g%3A'$ad_group'&writeaces=u%3A'$sec_username'' --user $username:$password
echo "## Reset the individual ACE on the volume. ##"
curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/modify?name='$username-$secvol_name'&writeAce=u%3A'$sec_username'' --user $username:$password

Now let's apply our policy we created to the volume:

In [ ]:
source ~/.bashrc
echo "## Apply the new security policy to your volume. ##"
curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/modify?name='$username-$secvol_name'&securitypolicy='$sec_policy'' --user $username:$password

In [ ]:
%logout

Now let's test that our other users can write files to the volume now:

You are logging in as the user:
<p align="center">
  <img src="Pictures/sec-studentxxxx.png" alt="studentxxx-Dev">                                                    
  
</p>

In [ ]:
%login sec-appliance

In [ ]:
source ~/.bashrc
echo "## Make an update to the file as the new user. ##"
echo "$sec_username-$secvol_name security volume sec user update $(date)" >> /mapr/$cluster_name/user/$username-$secvol_name/$username-security-volume-secuser-test1
echo "## List out the volume diretory to the files. ##"
ls /mapr/$cluster_name/user/$username-$secvol_name/
echo "## Verify the update to the file as the new user. ##"
cat /mapr/$cluster_name/user/$username-$secvol_name/$username-security-volume-secuser-test1



In [ ]:
%logout

### Let's cleanup now

 1. Remove Security Policies
 2. Remove Volumes

In [ ]:
%login {{ hostvars[inventory_hostname]['IP-WKSHP-Data-Fabric101'] }}


In [ ]:
source ~/.bashrc

#Remove Volume
curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/remove?name='$username-$secvol_name'' --user $username:$password

In [ ]:
%logout 

For more information on creating security policies please see [Creating Security Policies](https://docs.datafabric.hpe.com/62/SecurityGuide/CreatingSecurityPolicies.html) and [Managing File and Directory ACE's](https://docs.datafabric.hpe.com/62/SecurityGuide/FileDirACE.html)

## Using Access Security Settings and Policies Summary

In this example we:

1.	Created a new volume
2.	Tested writing and reading data to it from multiple users.
3.	Applied a ‘read only’ Access Control Expression to the volume.
4.	Tested writing and reading data to it from the sec-studentXXXX user.
5.	Created a security policy with read and write Access Control Expression’s.
6.	Applied the security policy to the volume.
7.	Tested writing and reading data to it from the sec-studentXXXX user.


# Time to wrap up
You can now move to the conclusion of this workshop !

* [Conclusion](5-WKSHP-Conclusion.ipynb)